<a href="https://colab.research.google.com/github/Sweetydutta8/Python-and-Machine-Learning/blob/master/Stakoverflow_tag_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stack Overflow is the largest, most trusted online community for developers to learn, share their programming knowledge, and build their careers.
Stack Overflow is something which every programmer use one way or another. Each month, over 50 million developers come to Stack Overflow to learn, share their knowledge, and build their careers. It features questions and answers on a wide range of topics in computer programming. The website serves as a platform for users to ask and answer questions.

Link to dataset :  https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction/

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import re
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import sparse
from wordcloud import WordCloud
from sqlalchemy import create_engine # database connection
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
DATAPATH = "../Data/"
DATA_DB = "TRAIN.db"
DATA_CSV = "TRAIN.csv"
DATA_DB_DEDUP = "TRAIN_NEW.db"
DATA_PROCESSED = "PROCESSED.DB"

In [ ]:
if not os.path.isfile(os.path.join(DATAPATH, DATA_DB)):
    start = datetime.now()
    #this sqlalchemy create_engine creates an in-memory SQLite database.
    engine = create_engine("sqlite:///"+os.path.join(DATAPATH, DATA_DB)) #https://docs.sqlalchemy.org/en/13/core/engines.html
    chunksize = 150
    cnt = 0
    for rows in pd.read_csv(os.path.join(DATAPATH, DATA_CSV), names = ["Id", "Title", "Body", "Tags"], chunksize = chunksize, iterator = True):
        cnt+=1
        rows.to_sql('Rows', engine, if_exists='append', index = False)
        if cnt % 1000 == 0:
            print("{} rows written".format(cnt*chunksize))
    print("Time taken to run this cell :", datetime.now() - start)
    
#Here above what we are doing is that, first we have created an in memory sqlite database using the line: 
#"create_engine("sqlite:///"+os.path.join(DATAPATH, DATA_DB))". 
#After this we are reading our csv file in chunks of 150 as an iterator. It simply means that whenever the loop runs we are 
#simply reading 150 rows everytime and appending this 150 rows in the sqlite database created above in the table name "Rows".

In [ ]:
if os.path.isfile(os.path.join(DATAPATH, DATA_DB)):
    start = datetime.now()
    connection = sqlite3.connect(os.path.join(DATAPATH, DATA_DB))
    data_count = pd.read_sql_query("SELECT COUNT(*) FROM Rows", connection)
    connection.close()
    print("Total number of rows in database = {}".format(data_count["COUNT(*)"][0]))
    print("Time taken to run this cell :", datetime.now() - start)

In [ ]:
if os.path.isfile(os.path.join(DATAPATH, DATA_DB)):
    start = datetime.now()
    connection = sqlite3.connect(os.path.join(DATAPATH, DATA_DB))
    data_dup = pd.read_sql_query("SELECT Title, Body, Tags, COUNT(*) as Count_Dup FROM Rows GROUP BY Title, Body, Tags", connection)
    connection.close()
    print("Time taken to run this cell : {}".format(datetime.now() - start))
#In this cell, this above sql command will group the title, body and tags where all of them are similar and then it will count
#their occurances.

In [ ]:
data_dup.head()

In [ ]:
print("Percentage of duplicate data points = {}({}%)".format((data_count["COUNT(*)"][0]-data_dup.shape[0]), (((data_count["COUNT(*)"][0]-data_dup.shape[0])/data_count["COUNT(*)"][0])*100)))

In [ ]:
data_dup['Count_Dup'].value_counts()

In [ ]:
if not os.path.isfile(os.path.join(DATAPATH, DATA_DB_DEDUP)):
    engine = create_engine("sqlite:///"+os.path.join(DATAPATH, DATA_DB_DEDUP))
    data_dup = pd.DataFrame(data_dup, columns=['Title', 'Body', 'Tags'])
    data_dup.to_sql("no_dup_train", engine)

In [ ]:
if os.path.isfile(os.path.join(DATAPATH, DATA_DB_DEDUP)):
    con = sqlite3.connect(os.path.join(DATAPATH, DATA_DB_DEDUP))
    data_tags = pd.read_sql_query("SELECT Tags FROM no_dup_train", con)
    data_tags_count = data_tags["Tags"].apply(lambda x: len(x.split(" "))) #series.apply(func) invoke function on values of Series.
    data_tags['Tags_Count'] = data_tags_count
    data_tags.head()

In [ ]:
print("Maximum number of tags per question = "+str(max(data_tags['Tags_Count'])))
print("Minimum number of tags per question = "+str(min(data_tags['Tags_Count'])))
print("Avg number of tags per question = "+str(sum(data_tags['Tags_Count'])/len(data_tags['Tags_Count'])))

In [ ]:
fig = plt.figure(figsize = (8, 6))
axes = fig.add_axes([0.1,0.1,1,1])
axes.set_title("Distribution of Tags per Question", fontsize = 20)
axes.set_xlabel("Tags", fontsize = 20)
axes.set_ylabel("Count", fontsize = 20)
plt.grid(linestyle='-', linewidth=0.5)
axes.tick_params(labelsize = 15)
sns.countplot(list(data_tags['Tags_Count']), ax = axes)
for i in axes.patches:
    axes.text(i.get_x()+0.09, i.get_height()+9500, str(round(i.get_height(), 2)), fontsize=16, color='black')
plt.show()

In [ ]:
data_tags['Tags_Count'].value_counts()

In [ ]:
#above cell shows that most of the data points have 3 tags then 2 tags and so on.






#Observations:

    #Maximum number of tags per question: 5
    #Minimum number of tags per question: 1
    #Avg. number of tags per question: 2.899
    #Most of the questions are having 2 or 3 tags

In [ ]:
if os.path.isfile(os.path.join(DATAPATH, DATA_DB_DEDUP)):
    con = sqlite3.connect(os.path.join(DATAPATH, DATA_DB_DEDUP))
    data_tags = pd.read_sql_query("SELECT Tags FROM no_dup_train", con)
    vectorizer = CountVectorizer(tokenizer = lambda x: x.split(" "))
    data_bow = vectorizer.fit_transform(data_tags['Tags'])
    con.close()

In [ ]:
print("Total number of datapoints = {}".format(data_bow.shape[0]))
print("Total number of unique tags = {}".format(data_bow.shape[1]))

In [ ]:
print("Some of the tags in our data: {}".format(vectorizer.get_feature_names()[:15]))

In [ ]:


#top 10 highest occurring tags
col_sum = data_bow.sum(axis = 0).A1 #data_bow.sum(axis = 0) will sum the column of sparse matrix then .A1 will convert that 
                                    #matrix into array.
feat_count = dict(zip(vectorizer.get_feature_names(), col_sum))
feat_count_sorted = dict(sorted(feat_count.items(), key = lambda x: x[1], reverse = True))
count_data = {"Tags":list(feat_count_sorted.keys()), "Count": list(feat_count_sorted.values())}
count_df = pd.DataFrame(data = count_data)
count_df[:10]



In [ ]:
worcloudPlot = WordCloud(background_color="black", width=1500, height=1000)
worcloudPlot.generate_from_frequencies(frequencies=feat_count)
plt.figure(figsize=(30,20))
plt.imshow(worcloudPlot, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
axes = count_df.head(20).plot(x = 'Tags', y = 'Count', kind = 'bar', figsize = (18, 10), fontsize = 15, grid = True)
plt.xlabel("")
plt.ylabel("Count", fontsize = 20)
plt.title("Top 20 Highest occurring Tags", fontsize = 20)
cnt = 0
for i in axes.patches:
    axes.text(i.get_x(), i.get_height()+2000, count_df.head(20)['Tags'][cnt], fontsize=12, color='black')
    cnt +=1


#Observations:

    #Majority of the most frequent tags are programming language.
    # C# is the top most frequent programming language.
    #Android, IOS, Linux and windows are among the top most frequent operating systems.

In [ ]:
frequency_tags = list(feat_count_sorted.values())

In [ ]:
plt.figure(figsize = (8, 6))
plt.plot(frequency_tags)
plt.title("Tag Numbers VS Frequency", fontsize=20)
plt.xlabel("Tag Numbers", fontsize=15)
plt.ylabel("Frequency", fontsize=15)
plt.grid(linestyle='-', linewidth=0.5)



Above plot shows that there are very few tags whose frequency is very high, however, there are many tags whose frequency is very low.
So, we will now plot the first 1000 most frequently occurring tags.


In [ ]:
plt.figure(figsize = (8, 6))
plt.plot(frequency_tags[:1000])
plt.title("Tag Numbers VS Frequency(First 1000)", fontsize=20)
plt.xlabel("Tag Numbers", fontsize=15)
plt.ylabel("Frequency", fontsize=15)
plt.grid(linestyle='-', linewidth=0.5)
print(frequency_tags[0:1000:25])



Even from above plot we are not be able to know how many tags are there whose occurrances are high?


In [ ]:
plt.figure(figsize = (8, 6))
plt.plot(frequency_tags[:100])
plt.title("Tag Numbers VS Frequency(First 100)", fontsize=20)
plt.xlabel("Tag Numbers", fontsize=15)
plt.ylabel("Frequency", fontsize=15)
plt.grid(linestyle='-', linewidth=0.5)

In [ ]:
print(frequency_tags[0:100])



From above plot we can easily observe that first 20 tags are occurring more than 50k times. 100th tag is occurring approx 13k times.


In [ ]:
fig = plt.figure(figsize = (10, 6))

axes = fig.add_axes([0.1,0.1,1,1])
axes.set_title("Quantile values of Tag Number VS Frequency", fontsize = 20)
axes.set_xlabel("Quantiles", fontsize = 20)
axes.set_ylabel("Frequency", fontsize = 20)
axes.plot(frequency_tags[0:100])

plt.scatter(x = np.arange(0, 100, 5), y = frequency_tags[0:100][::5], c = "blue", s = 70, label="quantiles with 0.05 intervals")
plt.scatter(x = np.arange(0, 100, 25), y = frequency_tags[0:100][::25], c = "red", s = 70, label="quantiles with 0.25 intervals")
plt.legend(loc='upper right', fontsize = 20)

for x, y in zip(np.arange(0, 100, 25), frequency_tags[0:100][::25]):
    plt.annotate(s = '({},{})'.format(x, y), xy = (x, y), fontweight='bold', fontsize = 16, xytext=(x-1, y+5500))
    
axes.tick_params(labelsize = 15)

In [ ]:
print("Number of tags occurring in more than 10k datapoints = {}".format(count_df[count_df['Count']>10000].shape[0]))
print("Number of tags occurring in more than 100k datapoints = {}".format(count_df[count_df['Count']>100000].shape[0]))


#Observations:

    #There are 153 tags which are occurring in more than 10k datapoints.
    #There are 14 tags which are occurring in more than 100k datapoints.
    #Most frequent tag is c# which is occurring in 331505 datapoints.